In [ ]:
from collections import namedtuple
import random

In [ ]:
import pandas as pd

In [ ]:
import orchid
import orchid.reference_origins as origins
import orchid.project_loader as opl

In [ ]:
import toolz.curried as toolz

In [ ]:
from Orchid.FractureDiagnostics.RatioTypes import ProppantConcentration, SlurryRate
import UnitsNet

In [ ]:
# Path name of whatever project you wish to load.
loader = opl.ProjectLoader(
    r'c:/src/Orchid.IntegrationTestData/Project-frankNstein_Montney_UTM13_METERS.ifrac')

In [ ]:
# The returned value from `native_project` is . NET project. NOTE: each instance
# of the `ProjectLoader` class loads **exactly one** .NET project. To load a 
# different project, you must create a new `ProjectLoader` instance.
native_project = loader.native_project()

In [ ]:
all_wells = native_project.Wells.Items

In [ ]:
well_of_interest = all_wells[2]  # 'Hor_03'

In [ ]:
well_of_interest.Name

In [ ]:
all_stages_for_well_of_interest = well_of_interest.Stages.Items

In [ ]:
len(all_stages_for_well_of_interest)

In [ ]:
all_stage_numbers = list(range((len(all_stages_for_well_of_interest))))
random.shuffle(all_stage_numbers)
all_stage_numbers

In [ ]:
sampled_stage_numbers = [0, 27, 7, 22]

In [ ]:
sampled_stage_number = 0

In [ ]:
stage_of_interest = all_stages_for_well_of_interest[sampled_stage_number]
treatment_curves_of_interest = {c.SampledQuantityName: c for c in stage_of_interest.TreatmentCurves.Items}

In [ ]:
samples_of_interest = toolz.valmap(lambda c: c.GetOrderedTimeSeriesHistory(), treatment_curves_of_interest)
samples_of_interest

In [ ]:
sample_lengths_of_interest = toolz.valmap(lambda c: len(c), samples_of_interest)
sample_lengths_of_interest

In [ ]:
def generate_sample(n):
    items = list(range(n))
    random.shuffle(items)
    return items[:8]

sample_indices_of_interest = toolz.valmap(generate_sample, sample_lengths_of_interest)
sample_indices_of_interest

In [ ]:
quantity_name_type_map = {'Pressure': 'pressure',
                          'Surface Proppant Concentration': 'proppant',
                          'Slurry Rate': 'slurry'}

quantity_name_unit_map = {
    'Pressure':  UnitsNet.Pressure.GetAbbreviation(native_project.ProjectUnits.PressureUnit),
    'Surface Proppant Concentration': 
        ProppantConcentration.GetAbbreviation(
            native_project.ProjectUnits.ProppantConcentrationUnit.Item1,
            native_project.ProjectUnits.ProppantConcentrationUnit.Item2),
    'Slurry Rate': 
        SlurryRate.GetAbbreviation(
            native_project.ProjectUnits.SlurryRateUnit.Item1,
            native_project.ProjectUnits.SlurryRateUnit.Item2),
}

@toolz.curry
def all_sampled_samples(samples, indices, quantity_name):
    result = [(n, str(s.Timestamp), s.Value)
              for (n, s) in [(n, samples[quantity_name][n])
               for n in indices[quantity_name]]]
    return result
sampled_samples = all_sampled_samples(samples_of_interest, sample_indices_of_interest)

@toolz.curry
def all_sampled_data(field, well, stage, quantity_name, treatment_samples):
    result = [{'field': field,
               'well': well.Name,
               'stage_no': stage.DisplayStageNumber,
               'type': quantity_name_type_map[quantity_name],
               'index': sample[0],
               'timestamp': sample[1],
               'value': f'{sample[2]} {quantity_name_unit_map[quantity_name]}'}
               for sample in treatment_samples]
    return result
sampled_data_for_field_well_stage = all_sampled_data('Montney', well_of_interest, stage_of_interest)

def sampled_data(quantity_name):
    result = sampled_data_for_field_well_stage(quantity_name, sampled_samples(quantity_name))
    return result

pressure_data = sampled_data('Pressure')
proppant_data = sampled_data('Surface Proppant Concentration')
slurry_data = sampled_data('Slurry Rate')

In [ ]:
def merge_values(values):
    return values

data = toolz.merge_with(merge_values,
                        *list(toolz.concatv(pressure_data, proppant_data, slurry_data)))

pd.DataFrame(dict(data))